# Lab 6: Text Mining

The goal of this lab is to work with textual data, applying data mining techniques.  The basic objective
of text mining concerns the discovery and extraction of relevant and valuable information from large
volumes of textual data. More precisely, we will focus on the `opinion mining` problem (also known as `sentiment analysis`), which refers to the use of natural language processing, text analysis and machine learning tools in order to identify and extract subjective information from text corpora.

Our goal is to identify `positive` and `negative` opinions in reviews expressed by natural language (i.e., text) about a specific product. Opinion mining can be useful in several ways.  It can help a company to evaluate the success of an ad campaign or of new product, determine which versions of a product or service are popular and identify which demographics like or dislike particular product features.

There are several challenges in the problem of opinion mining. The first one concerns the fact that a specific word used to describe a product can be considered either as positive or as negative depending the context or the product. Let's take as example a review about a laptop that contains the word "long". If a customer said that the battery life of the laptop was long, that would be a positive opinion. However,  if the customer said that the laptop's start-up time was long, that would be a negative opinion. This example indicates that an opinion system, trained to gather opinions on one type of product or product's feature, may not perform very well on another. A second challenge is that people do not always express their opinions the same way. Most traditional text processing relies on the fact that small differences between two pieces of text do not change the meaning very much. However, in opinion mining, "the laptop was great" is very different from "the laptop was not great". Lastly, people can combine contradictory statements in their reviews, where  both positive and negative comments are written in the same sentence. Although this is easy for a human to understand, it is more difficult for a computer to parse.

In the context of this lab, we will analyze reviews about movies (e.g., obtained from  <a href='http://www.imdb.com/'>`IMDb`</a>, an online database of information related to films and television programs). We will follow a text categorization (or classification) approach to infer if the review is positive or negative (i.e., classify a review text as positive or negative). Next, we briefly describe the reviews dataset that will be used and then we present the steps of the pipeline that need to be performed for the sentiment analysis task.

### Dataset Description
The dataset that will be used in the lab concerns movie reviews. As you may have observed, in movie review websites like IMDb, any user can write its own review for a particular movie or television show. For example, the following figure shows the review of the movie `Midnight in Paris` given by a user. As we see, the opinion of this user for the movie is positive.

<img src="figures/imdb_review.png"> 

Here, we will consider such a dataset, where each review has been characterized as `positive` or `negative`. The data is contained in the `movie_reviews.csv` file. The file is composed of $1959$ reviews (one per line) and each one has been characterized as positive or negative. Below you are given a function that loads the data and their labels from the disk.

In [1]:
import csv

def load_file():
    with open('movie_reviews.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',',quotechar='"')
        
        # Initialize lists for data and class labels
        data =[]
        labels = []
        # For each row of the csv file
        for row in reader:
            # skip missing data
            if row[0] and row[1]:
                data.append(row[0].decode('utf-8'))
                y_label = -1 if row[1]=='negative' else 1
                labels.append(y_label)
    return data,labels

Use the function defined above to load the data. Print the first review of the dataset and its label. Note that label $1$ corresponds to a positive review, while label $-1$ to a negative review.

In [2]:
#your code here

### Description of the Task and the Pipeline

Our goal is to build a system that, given the review text of a movie, it will be able to predict if the opinion of the user is positive or negative. We will treat the problem as a classification one, where the goal is predict the class label, i.e., positive or negative, for a given review. In the description that follows, we use the word "document" to refer to text reviews. The outline of the task is given in the following figure.

<img src="figures/outline.png"> 

The pipeline that typically is followed to deal with the problem is similar to the one applied in any classification problem; the goal is to learn the parameters of a classifier from a collection of training documents (i.e., those reviews that we know if they are positive or negative) and then to predict the class of unlabeled documents.  The first step in text categorization is to transform documents, which typically are strings of characters, into a representation suitable for the learning algorithm and the classification task. Here, we will employ the `Vector Space Model`, a spatial representation of text documents. In this model, each document is represented by a vector in a $n$-dimensional space, where each dimension corresponds to a term (i.e., word) from the overall vocabulary of the given document collection. More formally, let $\mathcal{D}=\{d_1, d_2, \ldots, d_m\}$ denote a collection of $m$ documents, and $\mathcal{T}=\{t_1, t_2,\ldots,t_n\}$ be the dictionary, i.e., the set of words in the corpus $\mathcal{D}$. As we will describe later, $\mathcal{T}$ is obtained by applying some standard natural language processing techniques, such as stop words removal and stemming. Each document $d_i \in \mathcal{D}$ is represented as a vector of term weights $d_i=\{w_{i,1}, w_{i,2}, \ldots, w_{i,n}\}$, where $w_{i,k}$ is the weight of term $k$ in document $d_i$. That way, data can be represented by the `Document-Term matrix` of size $m \times n$, where the rows correspond to documents and the columns to the different terms (i.e., features) of set $\mathcal{T}$. Additionally, each document $d_i$ is associated with a class label $y_i=\{+,-\}$ (i.e., positive/negative opinion) forming the class vector $y$, and the goal is to predict the class labels for a set of test documents. Based on this formulation, traditional classification algorithms (e.g., SVMs, Logistic Regression, Random Forests, etc.) can be applied to predict the category label of test documents.

Next, we describe the tasks that need to be performed for each part, providing also parts of Python code.

### Data preprocessing

Before applying any learning algorithm to the data, it is necessary to apply some preprocessing tasks as shown below:

- Remove punctuation marks (e.g. .  ,  ?  :  (  )  [  ]) and transform all characters to lowercase. This can be done using Python's <a href='http://www.nltk.org/'>`NLTK`</a> library.

- Remove stop words. These are words that are filtered out before processing any natural language data. This set of words does not offer information about the content of the document and typically corresponds to a set of commonly used words in any language. For example, in the context of a search engine, suppose that your search query is "how to categorize documents". If the search engine tries to find web pages that contain the terms "how", "to", "categorize", "documents", it will find many more pages that contain the terms "how" and "to" than pages that contain information about document categorization. This is happening because the terms "how" and "to" are  commonly used in the English language.

- The third preprocessing step is the one of <a href='http://en.wikipedia.org/wiki/Stemming'>`stemming`</a>, i.e., the process of reducing the words to their word stem or root. For example, a stemming algorithm reduces the words "fishing", "fished", and "fisher" to the root word, "fish". In this lab, we will use `Porter's` stemmer, contained in the `NLTK` library.

You are given the following function that is given a list of textual documents and performs data preprocessing.

In [3]:
import nltk
from nltk.stem.porter import *
import string

def data_preprocessing(data):    
    # List of stopwords
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
                 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
                 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
                 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
                 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
                 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
                 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
                 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
                 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
                 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
                 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
                 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

    for doc_id, text in enumerate(data):
    
        # Remove punctuation and lowercase
        punctuation = set(string.punctuation)    
        doc = ''.join([w for w in text.lower() if w not in punctuation])
        
        # Stopword removal
        doc = [w for w in doc.split() if w not in stopwords]  
        
        # Stemming
        stemmer = PorterStemmer()
        doc = [stemmer.stem(w) for w in doc] 
        
        # Covenrt list of words to one string
        doc = ' '.join(w for w in doc)
        data[doc_id] = doc       
    return data    

Use the function to preprocess the reviews of the dataset.

In [4]:
#your code here

### Feature extraction and the TF-IDF matrix

After applying the preprocessing step, the text data (i.e., all the possible documents-reviews) should be transformed to a format that will be used in the learning (i.e., classification) task. As we describe above, the data will be represented by the Document-Term matrix, where the rows correspond to the different documents of the collection (i.e. reviews) and the columns to the features, which in our case are the different terms (i.e., words). Here, we are interested to find relevant weighting criteria for the Document-Term matrix, i.e., assign a relevance score $w_{ij}$ to each term $t_j$ for each document $d_i$ as shown in the next figure.

<img src='figures/tfidf_matrix.png'>

We will consider the `Bag-of-Words` representation of the documents. In this case, the importance of a term $t \in \mathcal{T}$ within a document $d \in \mathcal{D}$ is based on the frequency $tf(t,d)$ of the term in the document (TF). Furthermore, terms that occur frequently in one document but rarely in the rest of the documents, are more likely to be relevant to the topic of the document. This is known as the inverse document frequency (IDF) factor, and is computed at the collection level. It is obtained by dividing the total number of documents by the number of documents containing the term, and then taking the logarithm of that quotient, as follows:

$$ idf(t, \mathcal{D}) = \log \frac{m}{| \{d \in \mathcal{D} : t \in d |\} } $$

where $m$ is the total number of documents in collection $\mathcal{D}$, and the denominator captures the number of documents that term $t$ appears. Having computed the TF and IDF metrics, we can combine them to get the TF-IDF score:

$$ tf\text{-}idf(t,d,\mathcal{D}) = tf(t,d) \times idf(t, \mathcal{D}) $$

This function captures the intuitions that (i) the more often a term occurs in a document, the more it is representative of its content, and (ii) the more documents a term occurs in, the less discriminating it is. Using the TF-IDF score of each term for each document, we can fill in the weights $w_{ij}$ of the Document-Term matrix (see figure above).

Your next task is to construct the TF-IDF matrix of the dataset. Use the <a href='http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html'>`TfidfVectorizer`</a> object provided by the `scikit-learn` library. Examine the size and the sparsity of this matrix (i.e., fraction of non-zero elements). What do you observe?

In [5]:
#your code here

### Model learning and prediction

After creating the TF-IDF matrix, the problem has been transformed to a typical classification task: the rows of the matrix correspond to the instances (i.e. reviews) and the columns to the features (i.e. terms). Recall that, for each document we also have class information, i.e. positive or negative review, contained in the `labels` list (this list was created during the data load step). In order to train a classification model, we will split the dataset (`tfidf_matrix`) into two sets: training and test. The training set will be used to learn the parameters of the classification model, that later will be applied to the test part in order to examine the accuracy of the model. Your next task is to split the TF-IDF matrix using the <a href='http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html'>`train_test_split`</a> function of `scikit-learn`. Set the size of the test set equal to $40\%$ of the whole dataset.

In [6]:
#your code here

The next step of the pipeline involves the selection of the appropriate learning (i.e. classification) algorithm for the problem (e.g. Logistic Regression, SVMs, Random Forests). Here, you can test the performance of different algorithms and choose the best one. We will use built-in implementations of the classification algorithms provided by `scikit-learn`. Typically, in `scikit-learn`, we follow three steps in order to apply a classification algorithm: (i) create an object of the classifier, (ii) fit the parameters of the classifier to the training data, and (iii) do predictions on the test data. Use the <a href='http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html'>`Logistic Regression`</a> object of `scikit-learn` to create a Logistic Regression classifier and predict the labels of the test instances.

In [7]:
#your code here

### Evaluation of the classification resutls

For the evaluation of the classification results, we will use the precision, recall and $F_1$-score (combination of precision and recall) defined as follows:

$$ \text{precision} = \dfrac{TP}{TP + FP}, ~~~~~~~~ \text{recall} = \dfrac{TP}{TP + FN}, ~~~~~~~~~  F_1=2 \cdot \dfrac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}} $$

The precision for a class is the number of true positives $TP$ (i.e. the number of items correctly labeled as belonging to the positive class) divided by the total number of elements labeled as belonging to the positive class (i.e. the sum of true positives $TP$ and false positives $FN$, which are items incorrectly labeled as belonging to the class). Recall in this context is defined as the number of true positives $TP$ divided by the total number of elements that actually belong to the positive class (i.e. the sum of true positives $TP$ and false negatives $FN$, which are items which were not labeled as belonging to the positive class but should have been).

We will also use the <a href='http://en.wikipedia.org/wiki/Receiver_operating_characteristic'>`ROC curve`</a> which is a graphical plot that illustrates the performance of a binary classification system as its discrimination threshold is varied. The curve is created by plotting the true positive rate $TPR$ against the false positive rate $FPR$ at various threshold settings. The $TPR$ defines how many correct positive results occur among all positive samples available during the test. $FPR$ on the other hand, defines how many incorrect positive results occur among all negative samples available during the test. The curve is defined in the 2-dimensional space (x-axis and y-axis min and max values: $0$ to $1$). The best possible prediction method would yield a point in the upper left corner or coordinate $(0,1)$ of the ROC space. Thus, the area under the ROC curve (auc) is also an important characteristic. 

We will use build-in tools of `scikit-learn` to do the evaluation. Run the following code that computes the precision, recall, $F_1$ score and also plots the ROC curve.

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline

print classification_report(labels_test, labels_predicted)
print "The accuracy score is {:.2%}".format(accuracy_score(labels_test, labels_predicted))
    
# Compute ROC curve and area under the curve
fpr, tpr, thresholds = roc_curve(labels_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

# Plot ROC curve
plt.clf()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")

What do you observe about the performance of the algorithm? Repeat the same experiment replacing the Logistic Regression classifier in the previous step with the Random Forest and SVM classifiers.

---

### Latent Semantic Analysis

To reduce the sparsity of the TF-IDF matrix, we will use `Latent Semantic Analysis` (LSA). LSA performs `Singular Value Decomposition` (SVD) on the TF-IDF matrix and then produces a low-rank approximation of the original matrix that corresponds to the representations of the documents in a new space. If $\mathbf{M}$ is the TF-IDF matrix ($m \times n$ size), its Singular Value Decomposition is defined as:

$$ \mathbf{M} = \mathbf{U} \mathbf{\Sigma} \mathbf{V^T} $$
 
where $\mathbf{U}$ is a $m \times m$ matrix which has as columns the eigenvectors of $\mathbf{M} \mathbf{M^T}$, $\mathbf{\Sigma}$ is a $m \times n$ diagonal matrix with the singular values of $\mathbf{M}$ in the diagonal (= square roots of $\mathbf{M} \mathbf{M^T}$ eigenvalues) and $\mathbf{V}$ is a $n \times n$ matrix which has as columns the eigenvectors of $\mathbf{M^T} \mathbf{M}$.

We then find a low-rank approximation $\mathbf{\widetilde{M}}$ of $\mathbf{M}$ using $r$ dimensions instead of the original $m$ and $n$, $r<m, r<n$. The approximation is given by:
      
$$ \mathbf{\widetilde{M}}_{m \times n} = \mathbf{U}_{m \times r} \mathbf{\Sigma}_{r \times r} \mathbf{V}^T_{r \times n} $$

Perform LSA to generate a low-rank approximation of the original TF-IDF matrix. Use the <a href='https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html'>`svd`</a> function of the NumPy library to automatically perform the SVD decomposition of the TF-IDF matrix.

In [11]:
#your code here

Split the reduced TF-IDF matrix using the <a href='http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html'>`train_test_split`</a> function of `scikit-learn`. Then, train a Logistic Regression classifier and predict the labels of the test instances. Finally, compute the precision, recall, $F_1$ score and generate the ROC curve.

In [9]:
#your code here

### Word Embeddings

An alternative solution to constructing the document matrix is to use word embeddings. A word embedding is a function that embeds words into a vector space. Given the vector representations of the words, we can project each document in the word embedding space as the centroid of its words, i.e. we add the vectors of the words present in the text and normalize the sum by the total number of words. For an input sequence of words $d$, its vector representation $\boldsymbol{\mu}$ (`mean vector`) is given by:

$$ \mu = \frac{1}{|d|} \sum_{w \in d} w $$

where $|d|$ is the cardinality of $d$, i.e. its number of words, and $w$ is the word embedding of a word present in the document. The assumption is that these vectors are able to capture the semantic meaning associated with the contexts of the documents, enabling us to measure how similar they are to each other.

In this lab, we adopt the `word2vec` model in order to obtain a dense representation of words. Specifically, the vector representations of words come from a publicly available model consisting of $300$-dimensional vectors trained on a Google News dataset of about $100$ billion words. We will use these embeddings to project the documents in the $300$-dimensional space. To import the embeddings, use the following code. Note that words in the model are not stemmed and that not all words present in the documents are also available on the model.

In [10]:
def load_embeddings():
    # Read word embeddings from file
    vocabulary = {}
    embeddings_list = []
    with open('model.txt','r') as f:
        reader=csv.reader(f,delimiter='\t')
        for w,e in reader:
            vocabulary[w] = len(vocabulary)
            embeddings_list.append(e)

    embeddings = np.zeros((len(vocabulary), 300))
    for i in range(len(vocabulary)):
        embeddings[i,:] = np.array(embeddings_list[i].split(), dtype='f')
        
    return vocabulary,embeddings

vocabulary,embeddings = load_embeddings()

Create a matrix where each row corresponds to a document and each document is represented by the centroid of its words.

In [11]:
#your code here

Split the generated matrix using the <a href='http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html'>`train_test_split`</a> function of `scikit-learn`. Then, train a Logistic Regression classifier and use it to predict the labels of the test instances. Finally, compute the precision, recall, $F_1$ score and generate the ROC curve.

In [12]:
#your code here